In [1]:
# Import liberaries and functions
import pandas as pd
import numpy as np

In [2]:
rf_rates = pd.read_csv("Data\\rf_rates.csv")

rf_rates["date"] = pd.to_datetime(rf_rates["date"])
rf_rates.set_index("date", inplace=True)

mktrf > Excess Return on the Market\
rf > Risk-Free Return Rate (One Month Treasury Bill Rate)

In [3]:
prices_df = pd.read_csv("Data\Prices.csv")
prices_df["Date"] = pd.to_datetime(prices_df["Date"])
prices_df.set_index('Date', inplace=True)
prices_df.columns

Index(['Instrument', 'CLOSEPRICE', 'VOLUME', 'COMPANYMARKETCAP',
       'TTLCMNSHARESOUT'],
      dtype='object')

In [4]:
# Calculate returns
prices_df["Return"] = prices_df.groupby("Instrument")["CLOSEPRICE"].pct_change(1)
prices_df["log_Return"] = np.log(prices_df["Return"] + 1)

In [5]:
Beta_df = pd.merge(
    left=prices_df,
    right=rf_rates,
    left_on='Date',
    right_index=True,
    how='left'
)

Beta_df["ex_Return"] = Beta_df["log_Return"] - Beta_df["rf"]

In [6]:
N = 30

Beta_df[f"mktrf_var_{N}"] = (
    Beta_df.groupby('Instrument')["mktrf"]
    .rolling(N, min_periods=N//2).var()
).values

Beta_df[f"covar_{N}"] = (
    Beta_df.groupby("Instrument")[["mktrf", "ex_Return"]]
    .rolling(N, min_periods=N//2).cov()
    .unstack()["mktrf"]["ex_Return"]
).values

Beta_df[f"Beta_{N}"] = Beta_df[f"covar_{N}"] / Beta_df[f"mktrf_var_{N}"]

In [7]:
N = 90

Beta_df[f"mktrf_var_{N}"] = (
    Beta_df.groupby('Instrument')["mktrf"]
    .rolling(N, min_periods=N//2).var()
).values

Beta_df[f"covar_{N}"] = (
    Beta_df.groupby("Instrument")[["mktrf", "ex_Return"]]
    .rolling(N, min_periods=N//2).cov()
    .unstack()["mktrf"]["ex_Return"]
).values

Beta_df[f"Beta_{N}"] = Beta_df[f"covar_{N}"] / Beta_df[f"mktrf_var_{N}"]

In [8]:
N = 250

Beta_df[f"mktrf_var_{N}"] = (
    Beta_df.groupby('Instrument')["mktrf"]
    .rolling(N, min_periods=N//2).var()
).values

Beta_df[f"covar_{N}"] = (
    Beta_df.groupby("Instrument")[["mktrf", "ex_Return"]]
    .rolling(N, min_periods=N//2).cov()
    .unstack()["mktrf"]["ex_Return"]
).values

Beta_df[f"Beta_{N}"] = Beta_df[f"covar_{N}"] / Beta_df[f"mktrf_var_{N}"]

In [9]:
# Abnormal Return = Actual Return - Expected Return 
# Estimation window: 10 days before the event day
Beta_df["AR"] = Beta_df["log_Return"] - (
    Beta_df["rf"] + Beta_df.groupby("Instrument")["Beta_90"].shift(10)*Beta_df["mktrf"]
)

In [10]:
# Cumulative AR - Event window: -5 and +10 days
# sum of values over the event window at the event day (.shift(-10))
components = (
    Beta_df.groupby("Instrument")[["log_Return", "rf", "mktrf"]]
    .rolling(16, min_periods=9).sum().groupby("Instrument").shift(-10)
)

Beta_df["CAR_10"] = (
    (components["log_Return"] - components["rf"]).values - 
    Beta_df.groupby("Instrument")["Beta_90"].shift(10).values * components["mktrf"].values)

In [11]:
# Cumulative AR - Event window: -5 and +20 days
# sum of values over the event window at the event day
components = (
    Beta_df.groupby("Instrument")[["log_Return", "rf", "mktrf"]]
    .rolling(26, min_periods=14).sum().groupby("Instrument").shift(-20)
)

Beta_df["CAR_20"] = (
    (components["log_Return"] - components["rf"]).values - 
    Beta_df.groupby("Instrument")["Beta_90"].shift(10).values * components["mktrf"].values)

In [12]:
# Cumulative AR - Event window: -5 and +30 days
# sum of values over the event window at the event day
components = (
    Beta_df.groupby("Instrument")[["log_Return", "rf", "mktrf"]]
    .rolling(36, min_periods=19).sum().groupby("Instrument").shift(-30)
)

Beta_df["CAR_30"] = (
    (components["log_Return"] - components["rf"]).values - 
    Beta_df.groupby("Instrument")["Beta_90"].shift(10).values * components["mktrf"].values)

In [13]:
Beta_df.reset_index(inplace=True)

Beta_df.drop(
    columns=[
        'CLOSEPRICE', 'VOLUME', 'COMPANYMARKETCAP', 'TTLCMNSHARESOUT', 'Return', 'log_Return', 'ex_Return',
        'mktrf', 'rf', 'mktrf_var_30', 'covar_30', 'mktrf_var_90', 'covar_90', 'mktrf_var_250', 'covar_250'
    ],
    inplace=True
)

In [14]:
Beta_df.isna().sum()

Date                0
Instrument          0
Beta_30        926407
Beta_90       1141443
Beta_250      1712088
AR            2180847
CAR_10        1267230
CAR_20        1329997
CAR_30        1392674
dtype: int64

In [15]:
Beta_df.to_csv("Data\Beta_AR.csv", index=False)

In [ ]:
# Cumulative AR - Event window: -5 and +5 days 
Beta_df["CAR_5"] = (
    Beta_df.groupby("Instrument")["AR"].rolling(11, min_periods=6).mean()
    .groupby("Instrument").shift(-5)
).values

In [ ]:
market_idx = pd.read_csv('Data\market_daily.csv')

market_idx["DATE"] = pd.to_datetime(market_idx["DATE"])
market_idx.set_index("DATE", inplace=True)

vwretd > Value-Weighted Return (includes distributions)\
vwretx > Value-Weighted Return (excluding dividends)\
ewretd > Equal-Weighted Return (includes distributions)\
ewretx > Equal-Weighted Return (excluding dividends)\
totval > Total Market Value\
totcnt > Total Market Count\
usdval > Market Value of Securities Used\
usdcnt > Count of Securities Used\
sprtrn > Return on S&P Composite Index\
spindx > Level on S&P Composite Index